# Estimating Non-Mandatory Tour Frequency

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd
import activitysim
activitysim.__version__

'1.1.0'

We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('test')

# Load data and prep model for estimation

In [3]:
modelname = "nonmand_tour_freq"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True, condense_parameters=True)

The prototype model spec we are re-estimating has 210 rows for each person type, but the
accompanying dataset is not large enough to successfully estimate anywhere near than many
parameters. The `condense_parameters` option is activated here as a short cut to making
a model that can be estimated with stable parameter results.  When activated, it merges
parameters not only by name (i.e. when the same name appears twice it is the same parameter)
but also by value, so that if the initial value of any two parameters is identical
then they are treated as the same parameter.  Using "condense_parameters" in actual model
estimation efforts is ill advised and may generate confusing or unexpected results.

This component actually has a distinct choice model for each person type, so
instead of a single model there's a `dict` of models.

In [4]:
type(model)

dict

In [5]:
model.keys()

dict_keys(['PTYPE_FULL', 'PTYPE_PART', 'PTYPE_UNIVERSITY', 'PTYPE_NONWORK', 'PTYPE_RETIRED', 'PTYPE_DRIVING', 'PTYPE_SCHOOL', 'PTYPE_PRESCHOOL'])

# Review data loaded from the EDB

We can review the data loaded as well, similarly there is seperate data 
for each person type.

## Coefficients

In [6]:
data.coefficients['PTYPE_FULL']

,value,constrain
coefficient_name,,
coef_escorting_tour,0.0000,T
coef_discretionary_tour,0.0000,T
coef_shopping_tour,0.0000,T
coef_maintenance_tour,0.0000,T
coef_visiting_or_social_tour,0.0000,T
...,...,...
coef_1_plus_maintenance_tours_constant,0.1202,F
coef_1_plus_eating_out_tours_constant,0.0097,F
coef_1_plus_visting_tours_constant,0.0522,F


## Utility specification

In [7]:
data.spec['PTYPE_FULL']

0                   coef_urban_and_discretionary_tour
1                   coef_urban_and_discretionary_tour
2                   coef_urban_and_discretionary_tour
3                   coef_urban_and_discretionary_tour
4                   coef_urban_and_discretionary_tour
                            ...                      
205            coef_1_plus_maintenance_tours_constant
206             coef_1_plus_eating_out_tours_constant
207                coef_1_plus_visting_tours_constant
208    coef_1_plus_other_discretionary_tours_constant
209          coef_0_auto_household_and_escorting_tour
Name: PTYPE_FULL, Length: 210, dtype: object

## Chooser data

In [8]:
data.chooser_data['PTYPE_FULL']

,person_id,model_choice,override_choice,household_id,age,PNUM,sex,pemploy,pstudent,ptype,...,high_income,no_cars,car_sufficiency,num_hh_joint_shop_tours,num_hh_joint_eatout_tours,num_hh_joint_maint_tours,num_hh_joint_social_tours,num_hh_joint_othdiscr_tours,has_mandatory_tour,has_joint_tour
0,72241,0,0,72241,56,1,1,1,3,1,...,False,False,0,0,0,0,0,0,1,0
1,72441,0,0,72441,49,1,1,1,3,1,...,False,False,0,0,0,0,0,0,1,0
2,73144,0,0,73144,31,1,2,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
3,73493,0,0,73493,31,1,2,1,3,1,...,False,False,0,0,0,0,0,0,1,0
4,73706,0,0,73706,26,1,1,1,3,1,...,False,False,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,7512288,5,5,2820953,28,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,0,0
1760,7512469,1,1,2821134,34,1,2,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
1761,7513117,0,0,2821782,16,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
1762,7513996,0,0,2822661,24,1,1,1,3,1,...,False,False,0,0,0,0,0,0,1,0


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  The prototype model spec we are re-estimating has 210 rows for each person type, but the accompanying dataset is not large enough to successfully estimate anywhere near than many parameters, so a short cut is applied by having one parameter only per unique existing parameter value.

In [9]:
for k, m in model.items():
    print(f"Person type {k} has {len(m.utility_ca)} utility terms and {len(m.pf)} unique parameters.")

Person type PTYPE_FULL has 210 utility terms and 72 unique parameters.
Person type PTYPE_PART has 210 utility terms and 51 unique parameters.
Person type PTYPE_UNIVERSITY has 210 utility terms and 70 unique parameters.
Person type PTYPE_NONWORK has 210 utility terms and 77 unique parameters.
Person type PTYPE_RETIRED has 210 utility terms and 53 unique parameters.
Person type PTYPE_DRIVING has 210 utility terms and 43 unique parameters.
Person type PTYPE_SCHOOL has 210 utility terms and 34 unique parameters.
Person type PTYPE_PRESCHOOL has 210 utility terms and 25 unique parameters.


For future estimation work, parameters can be intelligently named and applied to match the model developer's desired structure (by using the same named parameter for multiple rows of the spec file).  If this is done, the "short cut" should be disabled by setting `condense_parameters=False` in the loading step above.

Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [10]:
for k, m in model.items():
    m.estimate(method='SLSQP')

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,-2.0000,0.0,-2.0,-2.0,1,,-2.000000
coef_1_escort_tour_constant,0.319037,0.0298,0.0,NaN,NaN,0,,0.319037
coef_1_plus_eating_out_tours_constant,-1.012856,0.0097,0.0,NaN,NaN,0,,-1.012856
coef_1_plus_maintenance_tours_constant,-2.842643,0.1202,0.0,NaN,NaN,0,,-2.842643
coef_1_plus_other_discretionary_tours_constant,10.543979,0.7412,0.0,NaN,NaN,0,,10.543979
...,...,...,...,...,...,...,...,...
coef_walk_access_to_retail_and_discretionary,0.160520,0.0567,0.0,NaN,NaN,0,,0.160520
coef_walk_access_to_retail_and_eating_out,0.211289,0.1450,0.0,NaN,NaN,0,,0.211289
coef_walk_access_to_retail_and_escorting,-0.105285,0.0451,0.0,NaN,NaN,0,,-0.105285
coef_walk_access_to_retail_and_shopping,0.030182,0.0330,0.0,NaN,NaN,0,,0.030182


/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m.estimate(method='SLSQP')
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/linalg/__init__.py:18: UserWarning: minimum eig 5.456681257677148e-07 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model seems to have 3 parameter estimators with negative variance
- coef_1_escort_tour_constant
- coef_2_plus_escort_tours_constant
- coef_urban_and_escorting_tour
  m.estimate(method='SLSQP')
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: RuntimeWarning: invalid value encountered in sqrt
  m.estimate(method='SLSQP')
req_data does not request avail_ca or avail_co but it is s

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,-2.0000,0.0,-2.0,-2.0,1,,-2.000000
coef_1_escort_tour_constant,0.777927,0.5272,0.0,NaN,NaN,0,,0.777927
coef_1_plus_eating_out_tours_constant,1.665075,0.6914,0.0,NaN,NaN,0,,1.665075
coef_1_plus_maintenance_tours_constant,1.214734,0.5533,0.0,NaN,NaN,0,,1.214734
coef_1_plus_other_discretionary_tours_constant,1.783871,0.7989,0.0,NaN,NaN,0,,1.783871
coef_1_plus_shopping_tours_constant,1.521613,0.7569,0.0,NaN,NaN,0,,1.521613
coef_1_plus_visting_tours_constant,1.045482,0.1405,0.0,NaN,NaN,0,,1.045482
coef_2_plus_escort_tours_constant,1.882661,1.5987,0.0,NaN,NaN,0,,1.882661
coef_car_shortage_vs_workers_and_tour_frequency_is_5_plus,0.049232,-0.5498,0.0,NaN,NaN,0,,0.049232
coef_female_and_discretionary_tour,0.439252,0.3072,0.0,NaN,NaN,0,,0.439252


/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m.estimate(method='SLSQP')
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.818666298527715e-07 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model seems to have 2 parameter estimators with negative variance
- coef_1_escort_tour_constant
- coef_2_plus_escort_tours_constant
  m.estimate(method='SLSQP')
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: RuntimeWarning: invalid value encountered in sqrt
  m.estimate(method='SLSQP')
req_data does not request avail_ca or avail_co but it is set and being provided


...

/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m.estimate(method='SLSQP')
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/linalg/__init__.py:18: UserWarning: minimum eig 0.0 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model seems to have 14 parameter estimators with negative variance
- coef_1_plus_maintenance_tours_constant
- coef_1_plus_other_discretionary_tours_constant
- coef_1_plus_shopping_tours_constant
- coef_1_plus_visting_tours_constant
- and 10 more
  m.estimate(method='SLSQP')
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: RuntimeWarning: invalid value encountered in sqrt
  m.estimate(method='SL

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,-2.0000,0.0,-2.0,-2.0,1,,-2.000000
coef_1_escort_tour_constant,-1.572081,-0.7551,0.0,NaN,NaN,0,,-1.572081
coef_1_plus_eating_out_tours_constant,-2.222210,1.1145,0.0,NaN,NaN,0,,-2.222210
coef_1_plus_maintenance_tours_constant,-1.497135,-0.5060,0.0,NaN,NaN,0,,-1.497135
coef_1_plus_other_discretionary_tours_constant,0.090818,0.4634,0.0,NaN,NaN,0,,0.090818
coef_1_plus_shopping_tours_constant,1.474688,0.4783,0.0,NaN,NaN,0,,1.474688
coef_1_plus_visting_tours_constant,-1.792608,-0.4006,0.0,NaN,NaN,0,,-1.792608
coef_2_plus_escort_tours_constant,-2.059623,-0.0086,0.0,NaN,NaN,0,,-2.059623
coef_auto_access_to_retail_and_escorting,0.555108,0.0629,0.0,NaN,NaN,0,,0.555108
coef_high_income_group_and_eating_out_tour,-1.361980,-0.7010,0.0,NaN,NaN,0,,-1.361980


/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m.estimate(method='SLSQP')
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.064162929131762e-17 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model seems to have 3 parameter estimators with negative variance
- coef_1_escort_tour_constant
- coef_2_plus_escort_tours_constant
- coef_urban_and_escorting_tour
  m.estimate(method='SLSQP')
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: RuntimeWarning: invalid value encountered in sqrt
  m.estimate(method='SLSQP')
req_data does not request avail_ca or avail_co but it is s

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,-2.0000,0.0,-2.0,-2.0,1,,-2.000000
coef_1_escort_tour_constant,0.179651,0.3622,0.0,NaN,NaN,0,,0.179651
coef_1_plus_eating_out_tours_constant,0.204827,0.9612,0.0,NaN,NaN,0,,0.204827
coef_1_plus_maintenance_tours_constant,0.073169,0.6788,0.0,NaN,NaN,0,,0.073169
coef_1_plus_other_discretionary_tours_constant,1.133662,1.4935,0.0,NaN,NaN,0,,1.133662
coef_1_plus_shopping_tours_constant,0.785367,1.6919,0.0,NaN,NaN,0,,0.785367
coef_1_plus_visting_tours_constant,-0.011286,0.4424,0.0,NaN,NaN,0,,-0.011286
coef_2_plus_escort_tours_constant,1.744429,2.2219,0.0,NaN,NaN,0,,1.744429
coef_discretionary_tour,0.543162,0.9030,0.0,NaN,NaN,0,,0.543162
coef_escorting_tour,1.353509,2.4910,0.0,NaN,NaN,0,,1.353509


/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m.estimate(method='SLSQP')
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.8528068249273466e-08 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: PossibleOverspecification: WARNING: Model seems to have 14 parameter estimators with negative variance
- coef_1_escort_tour_constant
- coef_1_plus_eating_out_tours_constant
- coef_1_plus_maintenance_tours_constant
- coef_1_plus_shopping_tours_constant
- and 10 more
  m.estimate(method='SLSQP')
/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/ipykernel_48972/89088409.py:2: RuntimeWarning: invalid value encountered in sqrt
  m.estimate(method=

### Estimated coefficients

In [11]:
model['PTYPE_FULL'].parameter_summary()

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,NA,0.00,fixed value
coef_1_escort_tour_constant,0.319,NA,NA,[*],2.36,0.00,
coef_1_plus_eating_out_tours_constant,-1.01,124.,-0.01,,NA,0.00,
coef_1_plus_maintenance_tours_constant,-2.84,124.,-0.02,,NA,0.00,
coef_1_plus_other_discretionary_tours_constant,10.5,123.,0.09,,NA,0.00,
coef_1_plus_shopping_tours_constant,7.18,124.,0.06,,NA,0.00,
coef_1_plus_visting_tours_constant,-0.321,124.,-0.00,,NA,0.00,
coef_2_plus_escort_tours_constant,0.607,NA,NA,[],0.97,0.00,
coef_at_home_pre_driving_school_kid_and_escorting_tour,-0.926,1.05,-0.89,,NA,0.00,
coef_at_home_pre_school_kid_and_discretionary_tour,-0.656,0.743,-0.88,,NA,0.00,


# Output Estimation Results

In [12]:
from activitysim.estimation.larch import update_coefficients
for k, m in model.items():
    result_dir = data.edb_directory/k/"estimated"
    update_coefficients(
        m, data.coefficients[k], result_dir,
        output_file=f"{modelname}_{k}_coefficients_revised.csv",
        relabel_coef=data.relabel_coef.get(k),
    );

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [13]:
for k, m in model.items():
    result_dir = data.edb_directory/k/"estimated"
    m.to_xlsx(
        result_dir/f"{modelname}_{k}_model_estimation.xlsx", 
        data_statistics=False,
    )

/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/util/excel.py:523: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  xl = ExcelWriter(filename, engine='xlsxwriter_larch', model=model, **kwargs)
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/util/excel.py:523: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  xl = ExcelWriter(filename, engine='xlsxwriter_larch', model=model, **kwargs)
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/util/excel.py:523: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  xl = ExcelWriter(filename, engine='xlsxwriter_larch', model=model, **kwargs)
/Users/jeffnewman/LocalGit/asim-larch/activitysim-larch/conda-environments/AL-ENV/lib/python3.9/site-packages/larch/util/excel.py:523: Futu

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [14]:
result_dir = data.edb_directory/'PTYPE_FULL'/"estimated"
pd.read_csv(result_dir/f"{modelname}_PTYPE_FULL_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_escorting_tour,0.000000,T
1,coef_discretionary_tour,0.000000,T
2,coef_shopping_tour,0.000000,T
3,coef_maintenance_tour,0.000000,T
4,coef_visiting_or_social_tour,0.000000,T
...,...,...,...
205,coef_1_plus_maintenance_tours_constant,-2.842643,F
206,coef_1_plus_eating_out_tours_constant,-1.012856,F
207,coef_1_plus_visting_tours_constant,-0.320820,F
208,coef_1_plus_other_discretionary_tours_constant,10.543979,F
